In [1]:
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim
from torch.optim.lr_scheduler import StepLR
import torch.utils.data
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import pretrainedmodels
import pretrainedmodels.utils
from model import get_model, get_vgg, MySeNet
from dataset import FaceDataset
from train import train, validate

Change data_dir to the direction of the 'appa-real-release' dataset

Download with cmd:

wget http://158.109.8.102/AppaRealAge/appa-real-release.zip

unzip appa-real-release.zip

In [2]:
model = MySeNet()
path = 'model_dir'
data_dir = '../appa-real-release'
start_epoch = 0
checkpoint_dir = Path(path + '/checkpoint')
checkpoint_dir.mkdir(parents=True, exist_ok=True)
resume_path = None
tensorboard_dir = path + '/tensorboard'
opts = []
multi_gpu = False


img_size = 224
age_stddev = 1.0
batch_size = 40
learning_rate = 1e-3
step_size = 20
decay_rate = 0.4
num_epochs = 30
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
train_dataset = FaceDataset(data_dir, "train", img_size=img_size, augment=False, age_stddev=age_stddev)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

val_dataset = FaceDataset(data_dir, "valid", img_size=img_size, augment=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False)


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.L1Loss().to(device)

if resume_path:
    if Path(resume_path).is_file():
        print("=> loading checkpoint '{}'".format(resume_path))
        checkpoint = torch.load(resume_path, map_location="cpu")
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(resume_path, checkpoint['epoch']))
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    else:
        print("=> no checkpoint found at '{}'".format(resume_path))


model = model.to(device)

scheduler = StepLR(optimizer, step_size=step_size, gamma=decay_rate, last_epoch=start_epoch - 1)
best_val_mae = 1000
train_writer = None

if tensorboard_dir is not None:
    opts_prefix = "_".join(opts)
    train_writer = SummaryWriter(log_dir=tensorboard_dir + "/" + opts_prefix + "_train")
    val_writer = SummaryWriter(log_dir=tensorboard_dir + "/" + opts_prefix + "_val")

In [ ]:
model = model.to(device)
for epoch in range(start_epoch, num_epochs):
    # train
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, device, beta=0, gamma=0, gr_acc=3)

    # validate
    _,_,val_loss , val_mae = validate(val_loader, model, criterion, epoch, device, beta=0, gamma=0)

    if tensorboard_dir is not None:
        train_writer.add_scalar("loss", train_loss, epoch)
        val_writer.add_scalar("loss", val_loss, epoch)
        val_writer.add_scalar("mae", val_mae, epoch)

    # checkpoint
    if val_mae < best_val_mae:
        print(f"=> [epoch {epoch:03d}] best val mae was improved from {best_val_mae:.3f} to {val_mae:.3f}")
        model_state_dict = model.module.state_dict() if multi_gpu else model.state_dict()
        torch.save(
            {
                'epoch': epoch + 1,
                'arch': "resnet",
                'optimizer' : 'Adam',
                'learning_rate': learning_rate,
                'state_dict': model_state_dict,
                'optimizer_state_dict': optimizer.state_dict()
            },
            str(checkpoint_dir.joinpath("epoch{:03d}_{:.5f}_{:.4f}.pth".format(epoch, val_loss, val_mae)))
        )
        best_val_mae = val_mae
    else:
        print(f"=> [epoch {epoch:03d}] best val mae was not improved from {best_val_mae:.3f} ({val_mae:.3f})")

    # adjust learning rate
    scheduler.step()

print("=> training finished")
print(f"additional opts: {opts}")
print(f"best val mae: {best_val_mae:.3f}")

## Train Residual DEX

load your model or download this <a href='https://drive.google.com/file/d/1X3MNYGzPTOSGKAmUu7uBfueqkDBZHMfR/view?usp=sharing'> one</a> and store it in model_dir 

In [5]:
## load your model 

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MySeNet().to(device)
resume_path = 'models/model.pth'
checkpoint = torch.load(resume_path, map_location="cpu")
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [6]:
# test your model 

test_dataset = FaceDataset(data_dir, "test", img_size=img_size, augment=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
gt, preds,_, test_mae = validate(test_loader, model, None, 0, device)
test_mae

  0%|          | 0/50 [00:00<?, ?it/s]

4.676564091166784

In [7]:
def store_residuals(data_dir, model, device = 'cpu', img_size=224, batch_size=128 , augment=False, age_stddev=1):
    
    model = model.to(device)
    for stage in ['train', 'valid', 'test']:
        print ('storing ' , stage)
        dataset = FaceDataset(data_dir, stage, img_size=img_size, augment=False, age_stddev=age_stddev, ignore_corrupted=False)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=False)
        gt, preds,_, mae = validate(loader, model, None, 0, device)
        diffs = gt - preds

        # store to data_dir/filename_stage.csv
        csv_path = Path(data_dir).joinpath(f"gt_avg_{stage}.csv")
        df = pd.read_csv(str(csv_path))
        df['residual'] = diffs
        df.to_csv(Path(data_dir).joinpath(f"residuals_{stage}.csv"),
                  columns = ['file_name', 'residual'], index=False)
         

In [8]:
# if you havn't done it store the residuals of your model and load the data 

#store_residuals(data_dir, model, device = device, img_size=img_size,
                 #batch_size=batch_size,augment=False, age_stddev=age_stddev)

train_dataset = FaceDataset(data_dir, "train", img_size=img_size, augment=False, age_stddev=age_stddev, load_residuals=True)
train_loader_r = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

val_dataset = FaceDataset(data_dir, "valid", img_size=img_size, augment=False,load_residuals=True)
val_loader_r = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False)


train a new model to predict residuals

In [ ]:
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=step_size, gamma=decay_rate, last_epoch=start_epoch - 1)

classes = torch.arange(-50,51,1)

for epoch in range(start_epoch, num_epochs):
    # train
    train_loss, train_acc = train(train_loader_r, model, criterion, optimizer, epoch, 
                                 device, beta=0, gamma=0, gr_acc=3, classes =classes)

    # validate
    _,_,val_loss , val_mae = validate(val_loader_r, model, criterion, epoch,
                                     device, beta=0, gamma=0, classes=classes )

    if tensorboard_dir is not None:
        train_writer.add_scalar("loss", train_loss, epoch)
        val_writer.add_scalar("loss", val_loss, epoch)
        val_writer.add_scalar("mae", val_mae, epoch)

    # checkpoint
    if val_mae < best_val_mae:
        print(f"=> [epoch {epoch:03d}] best val mae was improved from {best_val_mae:.3f} to {val_mae:.3f}")
        model_state_dict = model.module.state_dict() if multi_gpu else model.state_dict()
        torch.save(
            {
                'epoch': epoch + 1,
                'arch': "resnet",
                'optimizer' : 'Adam',
                'learning_rate': learning_rate,
                'state_dict': model_state_dict,
                'optimizer_state_dict': optimizer.state_dict()
            },
            str(checkpoint_dir.joinpath("epoch{:03d}_{:.5f}_{:.4f}.pth".format(epoch, val_loss, val_mae)))
        )
        best_val_mae = val_mae
    else:
        print(f"=> [epoch {epoch:03d}] best val mae was not improved from {best_val_mae:.3f} ({val_mae:.3f})")

    # adjust learning rate
    scheduler.step()

print("=> training finished")
print(f"additional opts: {opts}")
print(f"best val mae: {best_val_mae:.3f}")

In [ ]:
test_dataset = FaceDataset(data_dir, "test", img_size=img_size, augment=False,load_residuals=True)
test_loader_r = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

gt_residuals, preds_residuals,_, test_mae = validate(test_loader_r, model, None, 0, device
                                                    , classes=classes)
print ('=> residual test mae: ', test_mae)



In [ ]:
new_preds = preds + preds_residuals
diff = new_preds - gt
mae = np.abs(diff).mean()
print ('mae residual dex on test set=>', mae)